# Open Street Maps Data Import and Analysis

## Intro

Data for Raleigh, North Carolina was downloaded from MapZen (https://mapzen.com/data/metro-extracts).

It was converted to JSON while being structured and corrected/normalized, then imported into mongodb.  Code for this processing is detailed in "02 - Code used for Data Wrangling and Exploration".

In [1]:
import re
from pymongo import MongoClient
client = MongoClient()
db = client.osm
collection = db.raleigh

## Problems encountered

There are certainly multiple problems or areas for improvement, but three issues that stuck out were:

1. Inconsistent Street Names
2. Questionable Validity of Education Ammenity Tags
3. Non-standardized Postal Codes

### Street Names

Aside from the expected standardization (such as "st" to "Street") that was required, some nodes had street names that were easily correctible but very specific.  This means they required manual checking (for example, consistency with Google Maps) and specific one-off corrections:

* "Meadowmont Village CIrcle" becomes "Meadowmont Village Circle"
* "LaurelcherryStreet" becomes "Laurel Cherry Street"
* "Garrett Driver" becomes "Garrett Drive"

Others had street names that could not be verified:

* "Triangle Family Practice"
*  Multiple similar names with no best choice:
    * "NC Highway 55 West"
    * "NC Highway 55"
    * "Highway West"
    * "Highway 55 West"
    * "Highway 55"
    * "US 55"


### Use of Amenity Tags for Education

After importing the data I took a look at a few of the "education" amenity tags ('university', 'college', and 'school') to see how the major colleges and universities in the area may be marked.

In [2]:
#Example Code: List of 'university' tags
pipeline = [
    {'$match':{'amenity':'university'}},
    {'$group':{'_id':'$name', 'count':{'$sum':1}}},
    {'$sort':{'count':-1}},
]
docs = collection.aggregate(pipeline)
for r in docs['result']:
    print(r)

{'_id': 'Duke University East Campus', 'count': 3}
{'_id': 'Duke University Medical Center', 'count': 1}
{'_id': 'Campbell University: Norman Adrian Wiggins School of Law', 'count': 1}
{'_id': "St. Augustine's University", 'count': 1}
{'_id': 'Duke University Central Campus', 'count': 1}
{'_id': 'JC Raulston Arboretum at NC State University', 'count': 1}
{'_id': 'North Carolina Central University', 'count': 1}
{'_id': 'Duke University West Campus', 'count': 1}
{'_id': 'North Carolina State University (Centennial Campus)', 'count': 1}
{'_id': None, 'count': 1}
{'_id': 'William Peace University', 'count': 1}
{'_id': 'Campbell University RTP Campus', 'count': 1}


These tags seem to be infrequently used (when a name is included) and are inconsistent when they are used.  According to the specificaitons (https://wiki.openstreetmap.org/wiki/Map_Features#Education):

* "university" indicates a university campus.
* "college" indicates a college campus or building
* "school" indicates a school and grounds.

While some tags are as expected ("Duke University East Campus" appears multiple times as "university"), there are issues.

* There is no name for 1 universitiy, 28 college, and 21 school entries.
* "Durham Tech Community College" appears as a "college" while "Durham Technical Community College" appears twice as a school.
* "Duke University" is listed as a "school" while various campus regions are listed as "university".

I believe that this inconsistency is at least partially to blame on the unclear documentation for these tags.

### Non-standardized Postal Codes

Most postal codes (6,564) were valid, but there were two main groups- 5-digit and 9-digit codes.  These were standardized to 5-digit codes.  Invalid codes (there were only 6) were removed.  Regular expressions were used to identify the kind of postal code and correct it as needed:

In [3]:
postcode_re = re.compile(r'^[0-9]{5}$')
extended_postcode_re = re.compile(r'^[0-9]{5}-[0-9]{4}$')

def correct_postcode(postcode):
    """Try to convert postcode to 5 digit int"""
    if extended_postcode_re.match(postcode): #strip extended postcode with "-####"
        postcode = postcode[0:5]
        return int(postcode)
    elif postcode_re.match(postcode): #normal 5 digit postcode
        return int(postcode)
    else:
        return None
    
#Examples:
print(correct_postcode('27713'))
print(correct_postcode('27603-1407'))
print(correct_postcode('2612-6401'))

27713
27603
None


## Overview of the Data

###Filesizes:

* XML: 486.2 MiB
* JSON: 562.6 MiB

###Summary info about imported data

In [4]:
total = collection.find().count()
total_nodes = collection.find({"type":"node"}).count()
total_ways = collection.find({"type":"way"}).count()

print("{:,} Total Records".format(total))
print("{:,} Nodes".format(total_nodes))
print("{:,} Ways".format(total_ways))

2,735,730 Total Records
2,524,259 Nodes
211,464 Ways


###Total Users and Top Users (by number of records)

In [5]:
pipeline = [
    {'$group':{'_id':'$created.user', 'count':{'$sum':1}}},
    {'$sort':{'count':-1}},
]
users = collection.aggregate(pipeline)['result']

print('Total Users: {:,}'.format(len(users)))
print('Top Users by number of records:')
for u in users[0:5]:
    print(u)

Total Users: 696
Top Users by number of records:
{'_id': 'jumbanho', 'count': 2139329}
{'_id': 'woodpeck_fixbot', 'count': 128144}
{'_id': 'yotann', 'count': 67765}
{'_id': 'JMDeMai', 'count': 63378}
{'_id': 'runbananas', 'count': 43244}


### Changesets
Changesets are groups of changes made by a single user over a short period of time.

In [6]:
pipeline = [
    {'$group':{'_id':{'changeset':'$created.changeset', 'user':'$created.user'}, 'count':{'$sum':1}}},
    {'$sort':{'count':-1}},
    {'$limit':5}
]
changesets = collection.aggregate(pipeline)['result']

print('Largest 5 changesets:')
for cs in changesets:
    print(cs)

Largest 5 changesets:
{'_id': {'changeset': '2453134', 'user': 'woodpeck_fixbot'}, 'count': 20133}
{'_id': {'changeset': '3553745', 'user': 'jumbanho'}, 'count': 19980}
{'_id': {'changeset': '3557826', 'user': 'jumbanho'}, 'count': 19861}
{'_id': {'changeset': '3558134', 'user': 'jumbanho'}, 'count': 19860}
{'_id': {'changeset': '3553606', 'user': 'jumbanho'}, 'count': 19849}


###Type of Fuel used for BBQ

In [7]:
pipeline = [
    {'$match':{'amenity':'bbq'}},
    {'$group':{'_id':'$fuel', 'count':{'$sum':1}}},
    {'$sort':{'count':-1}},
    {'$skip':1} #'None' - no fuel tag- are listed first because they are most common.  Skip keeping them.
]

docs = collection.aggregate(pipeline)
for r in docs['result']:
    print(r)

{'_id': 'charcoal', 'count': 4}
{'_id': 'gas', 'count': 1}


## Additional Ideas

### GNIS and TIGER Data
Some data was imported into OSM from GNIS (USGS Geographic Names Information System) and TIGER (Topologically Integrated Geographic Encoding and Referencing system).  This additional information may be useful for testing consistency. However, this data may be outdated and is a small fraction of the data overall:

In [8]:
#nodes with gnis
nodes_with = collection.find({'gnis':{'$exists':True}}).count()
nodes_without = collection.find({'gnis':{'$exists':False}}).count()
print("Fraction of Nodes with GNIS Data = {:.2%}".format(nodes_with/(nodes_with+nodes_without)))

#ways with tiger
ways_with = collection.find({'tiger':{'$exists':True}}).count()
ways_without = collection.find({'tiger':{'$exists':False}}).count()
print("Fraction of Ways with TIGER Data = {:.2%}".format(ways_with/(ways_with+ways_without)))

Fraction of Nodes with GNIS Data = 0.03%
Fraction of Ways with TIGER Data = 0.65%


In [9]:
#Number of Node references in ways with TIGER data
pipeline = [{'$match':{'tiger':{'$exists':True}}},
            {'$project':{'nodenum':{'$size':'$node_refs'}}},
            {'$group':{'_id':'', 'sum':{'$sum':'$nodenum'}}}
           ]
docs = collection.aggregate(pipeline)
tiger_noderefs = docs['result'][0]['sum']

print('{:,} total nodes ({:.2%}) are referenced by ways that have TIGER data.'.format(
        tiger_noderefs, tiger_noderefs/total_nodes))
print('However, this number may include duplicate references.')

232,715 total nodes (9.22%) are referenced by ways that have TIGER data.
However, this number may include duplicate references.


### Change in Dataset Over Time
How has the dataset built up over time?  Using the timestamp value it is possible to count the number of nodes or ways added over time- in this case by month.  It looks like January 2010 had the highest count for both types, but big months for 'Ways' were mostly in 2013 and big months for 'Nodes' were mostly in 2009.  This is probably due to large automated imports at those times.

In [10]:
#Group by first 7 characters of created timestamp
from collections import Counter
def get_monthly_count(collection, doctype):
    """Return the monthly count as a dict based on start of timestamp"""
    pipeline = [
        {'$match':{'type':doctype}},
        {'$group':{'_id':{'$substr':['$created.timestamp',0,7]}, 'count':{'$sum':1}}},
        {'$sort':{'count':-1}}
        ]
    counts = collection.aggregate(pipeline)['result']
    count_dict = Counter()
    for d in counts:
        count_dict[d['_id']] = d['count']
    return count_dict

node_counts = get_monthly_count(collection, 'node')
way_counts = get_monthly_count(collection, 'way')

print('Average Nodes per Month = {:,}'.format(int(sum(node_counts.values())/len(node_counts))))
print('Average Ways per Month = {:,}'.format(int(sum(way_counts.values())/len(way_counts))))

print('\nMonths with the most Nodes:\n','-'*30)
for m in node_counts.most_common(5):
    print("{} with {:,} Nodes".format(m[0],m[1]))
    
print('\nMonths with the most Ways:\n','-'*30)
for m in way_counts.most_common(5):
    print("{} with {:,} Ways".format(m[0],m[1]))

Average Nodes per Month = 28,684
Average Ways per Month = 2,458

Months with the most Nodes:
 ------------------------------
2010-01 with 1,267,448 Nodes
2009-11 with 271,430 Nodes
2009-07 with 144,939 Nodes
2009-08 with 135,390 Nodes
2009-09 with 46,642 Nodes

Months with the most Ways:
 ------------------------------
2010-01 with 89,284 Ways
2013-02 with 20,061 Ways
2013-01 with 7,724 Ways
2009-11 with 7,109 Ways
2013-04 with 5,135 Ways


### Possible Duplicate Nodes

It seems that many nodes are nearly identical- they differ only in the created timestamp, id, and ObjectId.  These seem to have been imported using automated methods since the user and changeset is the same for the duplicate records and the timestamp is either the same or very close (within 5 minutes).  It may be that they were created only to serve as points in various ways or relations.

These could likely be merged in order to reduce the size of the database without losing meaninful information as long as the other types of records are also checked for references to these nodes.

In [11]:
#How many nodes match at least one other node's position
pipeline = [
    {'$group':{'_id':'$pos', 'count':{'$sum':1}}},
    {'$match':{'count':{'$gt':1}}}
]
docs = collection.aggregate(pipeline, allowDiskUse=True)['result']

print("{:,} nodes do not have a unique position.".format(len(docs)))

3,094 nodes do not have a unique position.


In [12]:
#How many match within a single changeset
pipeline = [
    {'$group':{'_id':{'pos':'$pos','changeset':'$created.changeset'}, 'count':{'$sum':1}}},
    {'$match':{'count':{'$gt':1}}}
]
docs = collection.aggregate(pipeline, allowDiskUse=True)['result']

print("{:,} nodes do not have a unique position/changeset combo.".format(len(docs)))

5,982 nodes do not have a unique position/changeset combo.


In [13]:
#Look at one set of these as an example
pipeline = [
    {'$group':{'_id':{'pos':'$pos','changeset':'$created.changeset'}, 'count':{'$sum':1}}},
    {'$match':{'_id.pos':{'$exists':True}}},
    {'$sort':{'count':-1}},
    {'$limit':1}
]
docs = collection.aggregate(pipeline, allowDiskUse=True)

for r in docs['result']:
    pos = r['_id']['pos']
    print("-"*30,"\n",pos,"\n","-"*30,"\n")
    overlaps = collection.aggregate({'$match':{'pos':pos}})
    for o in overlaps['result']:
        print(o)

------------------------------ 
 [35.7877479, -78.8708045] 
 ------------------------------ 

{'_id': ObjectId('559dec3e943bb0b0cde5ec4d'), 'id': '195496249', 'type': 'node', 'pos': [35.7877479, -78.8708045], 'created': {'timestamp': '2013-10-04T18:14:14Z', 'uid': '1494110', 'changeset': '18183244', 'user': 'KristenK', 'version': '7'}}
{'_id': ObjectId('559dec6e943bb0b0cd08fd47'), 'id': '2482640856', 'type': 'node', 'pos': [35.7877479, -78.8708045], 'created': {'changeset': '18183244', 'uid': '1494110', 'user': 'KristenK', 'timestamp': '2013-10-04T18:13:55Z', 'version': '1'}}
{'_id': ObjectId('559dec6e943bb0b0cd08fd48'), 'id': '2482640858', 'type': 'node', 'pos': [35.7877479, -78.8708045], 'created': {'changeset': '18183244', 'uid': '1494110', 'user': 'KristenK', 'timestamp': '2013-10-04T18:13:55Z', 'version': '1'}}
{'_id': ObjectId('559dec6e943bb0b0cd08fd4a'), 'id': '2482640859', 'type': 'node', 'pos': [35.7877479, -78.8708045], 'created': {'timestamp': '2013-10-04T18:13:55Z', 'versio

# Conclusion

Overall the data seems to be mostly valid and uniform, but there is room for improvement.  The size of the dataset could certainly be reduced by removing redundant information like duplicate nodes (same position no differing tags), but this would be a large undertaking.  The dataset shouldn't be considered anywhere near complete when it comes to annotations that aren't standard.  For example, most nodes described as a BBQ restaurant ('amenity' of 'bbq') do not list the type of fuel used.